# 1차 미니프로젝트(멜론 웹스크래핑)

In [154]:
#라이브러리 임포트
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests

## 멜론 랭킹 탑100 가져오기(BeautifulSoup사용)

In [182]:
# URL 가져오기
url = f"https://www.melon.com/chart/index.htm"

In [183]:
driver.implicitly_wait(10)
driver.get(url)

In [184]:
# 해당 웹페이지 html 정보 request 하기
response = requests.get(url, headers={"user-agent": "Mozilla/5.0"})
# 해당 페이지 DataFrame 으로 가져오기
table = pd.read_html(response.text)

In [185]:
# BeautifulSoup 파싱 라이브러리를 사용하면 손쉽게 html 파일을 파싱(정리)
html = bs(response.text, 'html.parser')

In [186]:
# html tbody에 랭킹 정보 담긴 것 확인.
# select_one() 이용해 select된 첫 번째 element를 한 개 리턴
tbody= html.select_one("#frm > div > table > tbody")

In [187]:
#데이터들 수용할 리스트(melon_list)생성
melon_list = []
cols = table[0].columns

In [188]:
#순위, 제목, 가수, 앨범 값 가져오기
for song in songs:
    rank = song.select("div > span.rank")[0].text
    title = song.select("div.ellipsis.rank01 > span > a")[0].text
    singer = song.select(" div.ellipsis.rank02 > a")[0].text
    album = song.select("div.ellipsis.rank03 > a")[0].text

    # "좋아요" 수 가져오기
    # 좋아요 수는 고정 변수가 아닌 유동 변수입니다. 따라서 좋아요 수를 json파일로 따로 추출.
    #song_id는 노래 마다 고정값이므로 좋아요 수 데이터를 가져오기에 필요합니다.
    song_id = song["data-song-no"]

    like_url = f"https://www.melon.com/commonlike/getSongLike.json?contsIds={song_id}"
    like_response = requests.get(like_url, headers={"user-agent": "Mozilla/5.0"})
    like_number = like_response.json()["contsLike"]
    like_dict = {}

    for like in like_number:
        likes = like["SUMMCNT"]
        like_dict[song_id] = likes

    temp_dict={}
    temp_dict["순위"] = rank
    temp_dict["제목"] = title
    temp_dict["가수"] = singer
    temp_dict["앨범명"] = album
    temp_dict["좋아요 수"] = likes
    
    melon_list.append(temp_dict)

In [189]:
# 데이터프레임으로 변환하기
df = pd.DataFrame.from_dict(melon_list)
display(df)

,순위,제목,가수,앨범명,좋아요 수
0,1,새삥 (Prod. ZICO) (Feat. 호미들),지코 (ZICO),스트릿 맨 파이터(SMF) Original Vol.3 (계급미션),85907
1,2,After LIKE,IVE (아이브),After LIKE,169168
2,3,Attention,NewJeans,NewJeans 1st EP 'New Jeans',166584
3,4,Shut Down,BLACKPINK,BORN PINK,81719
4,5,Hype boy,NewJeans,NewJeans 1st EP 'New Jeans',137187
...,...,...,...,...,...
95,96,인생찬가,임영웅,IM HERO,40886
96,97,사랑해 진짜,임영웅,IM HERO,41398
97,98,팡파레,다비치,Season Note,36861
98,99,가을밤에 든 생각,잔나비,잔나비 소곡집 l,106648


In [190]:
# 위의 데이터셋 CSV 파일로 저장하기 
df.to_csv("MelonTop100", index = False, encoding="utf-8-sig")

In [141]:
pd.read_csv("MelonTop100")

,순위,제목,가수,앨범명,좋아요 수
0,1,새삥 (Prod. ZICO) (Feat. 호미들),지코 (ZICO),스트릿 맨 파이터(SMF) Original Vol.3 (계급미션),85889
1,2,After LIKE,IVE (아이브),After LIKE,169165
2,3,Attention,NewJeans,NewJeans 1st EP 'New Jeans',166579
3,4,Shut Down,BLACKPINK,BORN PINK,81713
4,5,Hype boy,NewJeans,NewJeans 1st EP 'New Jeans',137180
...,...,...,...,...,...
95,96,인생찬가,임영웅,IM HERO,40886
96,97,사랑해 진짜,임영웅,IM HERO,41398
97,98,팡파레,다비치,Season Note,36861
98,99,가을밤에 든 생각,잔나비,잔나비 소곡집 l,106644


## 좋아요 수 가져오기(Selenium 사용)

In [191]:
from selenium import webdriver

In [192]:
from selenium import webdriver
driver = webdriver.Chrome('C:\pydata\chromedriver_win32\chromedriver.exe')

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  


In [193]:
driver.implicitly_wait(10)
driver.get(url)

# 받아온 페이지 정보를 를 chart라는 변수에 저장합니다.
chart=driver.page_source
html = bs(chart, 'html.parser')

In [194]:
like = html.select('tr td div.wrap button span.cnt')
print(like)

[<span class="cnt">
<span class="none">총건수</span>
85,907</span>, <span class="cnt">
<span class="none">총건수</span>
169,168</span>, <span class="cnt">
<span class="none">총건수</span>
166,584</span>, <span class="cnt">
<span class="none">총건수</span>
81,719</span>, <span class="cnt">
<span class="none">총건수</span>
137,187</span>, <span class="cnt">
<span class="none">총건수</span>
136,953</span>, <span class="cnt">
<span class="none">총건수</span>
203,262</span>, <span class="cnt">
<span class="none">총건수</span>
49,082</span>, <span class="cnt">
<span class="none">총건수</span>
40,255</span>, <span class="cnt">
<span class="none">총건수</span>
115,447</span>, <span class="cnt">
<span class="none">총건수</span>
84,754</span>, <span class="cnt">
<span class="none">총건수</span>
102,543</span>, <span class="cnt">
<span class="none">총건수</span>
95,416</span>, <span class="cnt">
<span class="none">총건수</span>
115,353</span>, <span class="cnt">
<span class="none">총건수</span>
181,688</span>, <span class="cnt">
<span class

In [195]:
no_like = []
#좋아요 숫자를 담기 위해 빈 리스트를 만든다
for likes in like:
    a=likes.text
    no_like.append(a)
    print(a)


총건수
85,907

총건수
169,168

총건수
166,584

총건수
81,719

총건수
137,187

총건수
136,953

총건수
203,262

총건수
49,082

총건수
40,255

총건수
115,447

총건수
84,754

총건수
102,543

총건수
95,416

총건수
115,353

총건수
181,688

총건수
28,603

총건수
27,497

총건수
20,269

총건수
160,118

총건수
215,542

총건수
72,358

총건수
124,401

총건수
69,934

총건수
42,624

총건수
176,943

총건수
70,964

총건수
159,704

총건수
122,296

총건수
18,215

총건수
116,656

총건수
77,692

총건수
85,106

총건수
235,041

총건수
145,292

총건수
117,875

총건수
92,494

총건수
72,206

총건수
152,445

총건수
77,747

총건수
43,222

총건수
284,234

총건수
37,674

총건수
31,077

총건수
29,483

총건수
236,762

총건수
35,475

총건수
404,984

총건수
62,133

총건수
342,517

총건수
61,510

총건수
290,106

총건수
209,056

총건수
34,649

총건수
204,826

총건수
161,759

총건수
201,369

총건수
140,867

총건수
38,757

총건수
127,554

총건수
27,375

총건수
441,137

총건수
49,328

총건수
139,791

총건수
160,011

총건수
26,648

총건수
76,153

총건수
124,429

총건수
252,373

총건수
90,125

총건수
140,487

총건수
84,144

총건수
50,662

총건수
213,790

총건수
131,768

총건수
37,097

총건수
33,365

총건수
76,776

총건수
185,813

총건수
277,734

총건수
32,119

In [196]:
name = []
search = '\n총건수\n'
for i in k:
    #replace를 사용하여 반복되는 \n총건수\n을 지웁니다.
    hong = i.replace('\n총건수\n', '')
    name.append(hong)
name

['85,904',
 '169,166',
 '166,582',
 '81,718',
 '137,186',
 '136,952',
 '203,261',
 '49,082',
 '40,251',
 '115,447',
 '84,753',
 '102,542',
 '95,415',
 '115,353',
 '181,687',
 '28,602',
 '27,496',
 '20,269',
 '160,118',
 '215,541',
 '72,358',
 '124,400',
 '69,933',
 '42,624',
 '176,943',
 '70,964',
 '159,704',
 '122,296',
 '18,214',
 '116,656',
 '77,691',
 '85,106',
 '235,040',
 '145,292',
 '117,875',
 '92,494',
 '72,206',
 '152,445',
 '77,747',
 '43,222',
 '284,234',
 '37,674',
 '31,077',
 '29,483',
 '236,762',
 '35,475',
 '404,984',
 '62,132',
 '342,517',
 '61,510',
 '290,106',
 '209,056',
 '34,649',
 '204,826',
 '161,759',
 '201,369',
 '140,867',
 '38,757',
 '127,554',
 '27,375',
 '441,137',
 '49,328',
 '139,791',
 '160,011',
 '26,647',
 '76,153',
 '124,429',
 '252,373',
 '90,125',
 '140,487',
 '84,144',
 '50,662',
 '213,790',
 '131,768',
 '37,097',
 '33,365',
 '76,776',
 '185,813',
 '277,734',
 '32,119',
 '196,080',
 '325,633',
 '182,028',
 '552,854',
 '43,279',
 '45,298',
 '213,099